<a href="https://colab.research.google.com/github/sue-bin/DA-session-quest/blob/master/1031_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification 과제
- San Francisco에서 일어난 범죄 관련 데이터 입니다. 설명변수들을 해석해서 범죄의 category를 분류하시면 됩니다~
- 캐글 competition 데이터 입니다. 혹시라도 FE에서 어려움을 겪으신다면 https://www.kaggle.com/c/sf-crime/overview 를 살짝 참고해보세용

In [175]:
import pandas as pd
import numpy as np
from collections import Counter
import datetime
from sklearn import metrics

In [176]:
df = pd.read_csv('/content/drive/My Drive/YBIGTA/DA 교육세션/2020-10-31 ML  Classification/과제/sf_crime.csv')
df.head(10)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541
5,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM UNLOCKED AUTO,Wednesday,INGLESIDE,NONE,0 Block of TEDDY AV,-122.403252,37.713431
6,2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,INGLESIDE,NONE,AVALON AV / PERU AV,-122.423327,37.725138
7,2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,BAYVIEW,NONE,KIRKWOOD AV / DONAHUE ST,-122.371274,37.727564
8,2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,RICHMOND,NONE,600 Block of 47TH AV,-122.508194,37.776601
9,2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,CENTRAL,NONE,JEFFERSON ST / LEAVENWORTH ST,-122.419088,37.807802


- Dates - 범죄 발생 timestamp
- Category - 범죄 사건의 카테고리 (이게 target variable 입니다!)
- Descript - 범죄 사건에 대한 자세한 설명
- DayOfWeek - 무슨 요일에 발생했는지
- PdDistrict - 관할 경찰서
- Resolution - 범죄 사건이 어떻게 처리되었는지
- Address - 범죄 발생이 일어난 위치와 가장 가까운 거리
- X - 경도
- Y - 위도

## 1. 데이터 형태 확인 및 전처리

In [177]:
df.shape

(878049, 9)

In [178]:
df.dtypes

Dates          object
Category       object
Descript       object
DayOfWeek      object
PdDistrict     object
Resolution     object
Address        object
X             float64
Y             float64
dtype: object

In [179]:
df.isnull().sum()

Dates         0
Category      0
Descript      0
DayOfWeek     0
PdDistrict    0
Resolution    0
Address       0
X             0
Y             0
dtype: int64

In [180]:
df.columns[[1,2,4,5]]

Index(['Category', 'Descript', 'PdDistrict', 'Resolution'], dtype='object')

In [181]:
df["Category"].unique()

array(['WARRANTS', 'OTHER OFFENSES', 'LARCENY/THEFT', 'VEHICLE THEFT',
       'VANDALISM', 'NON-CRIMINAL', 'ROBBERY', 'ASSAULT', 'WEAPON LAWS',
       'BURGLARY', 'SUSPICIOUS OCC', 'DRUNKENNESS',
       'FORGERY/COUNTERFEITING', 'DRUG/NARCOTIC', 'STOLEN PROPERTY',
       'SECONDARY CODES', 'TRESPASS', 'MISSING PERSON', 'FRAUD',
       'KIDNAPPING', 'RUNAWAY', 'DRIVING UNDER THE INFLUENCE',
       'SEX OFFENSES FORCIBLE', 'PROSTITUTION', 'DISORDERLY CONDUCT',
       'ARSON', 'FAMILY OFFENSES', 'LIQUOR LAWS', 'BRIBERY',
       'EMBEZZLEMENT', 'SUICIDE', 'LOITERING',
       'SEX OFFENSES NON FORCIBLE', 'EXTORTION', 'GAMBLING', 'BAD CHECKS',
       'TREA', 'RECOVERED VEHICLE', 'PORNOGRAPHY/OBSCENE MAT'],
      dtype=object)

In [182]:
for col in list(df.columns[[1,2,4,5]]):
  print("column: ",col)
  print("# of unique values: ",len(list(df[col].unique())))

column:  Category
# of unique values:  39
column:  Descript
# of unique values:  879
column:  PdDistrict
# of unique values:  10
column:  Resolution
# of unique values:  17


## 2. Feature Engineering 진행
- Insight 하나) 위도와 경도를 하나로 합쳐줄 수 있지 않을까요?

In [184]:
#위도 경도 하나의 어레이로 결합
df['(X,Y)'] = [str((round(x,3),round(y,3))) for x, y in zip(df.X, df.Y)] #가까운 지역들끼리 라벨링 될 수 있도록 반올림(address와 너무 상관 높을것같긴함)
df.drop(["X","Y"],axis=1,inplace=True)

In [185]:
#dates 타입 datetime으로 변환
df['Dates'] = pd.to_datetime(df['Dates'], format="%Y-%m-%d %H:%M:%S")

In [186]:
df.dtypes

Dates         datetime64[ns]
Category              object
Descript              object
DayOfWeek             object
PdDistrict            object
Resolution            object
Address               object
(X,Y)                 object
dtype: object

In [187]:
#dates 에서는 정확한 날짜보다 연도, 월, 시간대 정도가 의미있을 것 같아 따로 열로 추가한 후 dates 제거
df["year"] = df["Dates"].apply(lambda x: x.year)
df["month"] = df["Dates"].apply(lambda x: x.month)
df["hour"] = df["Dates"].apply(lambda x: x.hour)
df.drop("Dates",axis=1,inplace=True)

In [188]:
df

,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,"(X,Y)",year,month,hour
0,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,"(-122.426, 37.775)",2015,5,23
1,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,"(-122.426, 37.775)",2015,5,23
2,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,"(-122.424, 37.8)",2015,5,23
3,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,"(-122.427, 37.801)",2015,5,23
4,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,"(-122.439, 37.772)",2015,5,23
...,...,...,...,...,...,...,...,...,...,...
878044,ROBBERY,ROBBERY ON THE STREET WITH A GUN,Monday,TARAVAL,NONE,FARALLONES ST / CAPITOL AV,"(-122.459, 37.714)",2003,1,0
878045,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,INGLESIDE,NONE,600 Block of EDNA ST,"(-122.447, 37.732)",2003,1,0
878046,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,SOUTHERN,NONE,5TH ST / FOLSOM ST,"(-122.403, 37.78)",2003,1,0
878047,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,SOUTHERN,NONE,TOWNSEND ST / 2ND ST,"(-122.391, 37.781)",2003,1,0


## 3. 범주형 변수들 더미화

In [189]:
from sklearn.preprocessing import LabelEncoder

categorical = list(df.columns[df.dtypes=="object"])
print(categorical)

enc = LabelEncoder()
for i in categorical:
    df[i] = enc.fit_transform(df[i])
    df[i] = df[i].astype('float64')

['Category', 'Descript', 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address', '(X,Y)']


## 4. Train/Test Split

In [190]:
from sklearn.model_selection import train_test_split, cross_val_score
y = df["Category"]
x = df.drop("Category",axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=101)

In [191]:
print(x_train.shape)
print(x_test.shape)

(658536, 9)
(219513, 9)


## 5. Decision Tree에 넣고 결과 확인, Feature Importance 까지 확인해보기

In [192]:
from sklearn.tree import DecisionTreeClassifier
dct = DecisionTreeClassifier()
dct = dct.fit(x_train, y_train)
y_pred = dct.predict(x_test)
print(dct.score(x_train, y_train))
print(dct.score(x_test, y_test)) #Descript와의 연관성이 높아서 높은 점수 나오는듯

0.9999772222019753
0.9993986688715475


In [193]:
print(dct.feature_importances_) #descript의 중요도가 제일 높음

[9.48733227e-01 4.58417178e-05 1.40557214e-04 3.96345043e-02
 1.23387621e-04 1.32025224e-04 1.10472589e-02 9.25761726e-05
 5.06223066e-05]


## 6. Random Forest, LGBM, XGBoost 중 하나에 넣고 결과 확인

In [ ]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(n_estimators = 500, max_depth=10)
lgbm.fit(x_train, y_train)
y_pred = lgbm.predict(x_test)
lgbm.score(x_test, y_test)

### 어떤 알고리즘을 사용하셨나요? 왜 그 알고리즘을 사용하셨나요?

lightgbm 사용. data 크기가 커서 더 학습시간이 짧은 모델 사용